In [39]:
import random
import openai
from semantic_text_splitter import TextSplitter
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

client = OpenAI()


def split_and_merge_text(text, min_size=2500, max_size=3000):
    """
    Splits the text using a TextSplitter and merges smaller chunks to meet size constraints.

    Args:
        text (str): The input text to be split.
        min_size (int): The minimum size for each chunk.
        max_size (int): The maximum size for each chunk.

    Returns:
        list: A list of merged chunks that meet the size constraints.
    """
    # Initialize the splitter with the chunk size range
    splitter = TextSplitter((min_size, max_size))

    # Split the text into initial chunks
    initial_chunks = splitter.chunks(text)

    # Merge chunks smaller than the minimum size
    merged_chunks = []
    current_chunk = ""
    for chunk in initial_chunks:
        if len(current_chunk) + len(chunk) <= max_size:
            current_chunk += chunk
        else:
            if current_chunk:
                merged_chunks.append(current_chunk)
            current_chunk = chunk

    if current_chunk:
        merged_chunks.append(current_chunk)

    # Ensure all chunks meet the minimum size requirement
    final_chunks = []
    current_chunk = ""
    for chunk in merged_chunks:
        if len(chunk) < min_size:
            current_chunk += chunk
        else:
            if current_chunk:
                final_chunks.append(current_chunk)
                current_chunk = ""
            final_chunks.append(chunk)

    if current_chunk:
        final_chunks.append(current_chunk)

    return final_chunks

def extract_title_and_author(text):
    """
    Extracts the title and author from the first few lines of a given text.

    Args:
        text (str): The input text containing title and author information.

    Returns:
        tuple: A tuple containing the title and author as strings.
    """
    title = None
    author = None

    # Split the text into lines and check the first few lines
    lines = text.splitlines()
    for line in lines[:5]:  # Look at the first few lines for these fields
        if line.startswith("Titre :"):
            title = line.split(":", 1)[1].strip()
        elif line.startswith("Auteur :"):
            author = line.split(":", 1)[1].strip()

    return title, author

# Function to retrieve a specific insight from a chunk
def retrieve_insight(chunk, author, title):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": (
                "You are an analytical and thought-provoking assistant specializing in extracting deep insights from social science texts. "
                "Your goal is to identify nuanced perspectives, uncover underlying themes, and present information in an engaging and intellectually stimulating manner. "
                "Focus on creating clear, concise, and structured breakdowns of ideas, avoiding mainstream entertainment references."
            )},
            {
                "role": "user",
                "content": (
                    f"""Le texte suivant est extrait de "{title}", écrit par {author}.
                    En mettant l'accent sur la perspective de {author}, identifie l'idée générale qu'il exprime selon son point de vue.
                    Ensuite, décompose cette idée en étapes successives, chacune correspondant à une phrase courte.
                    Chaque phrase doit décrire une étape claire et concise pour arriver à l'idée générale, tout en s'appuyant sur des détails du texte.
                    Introduit chaque étape par "1.", "2."...
                    Si une phrase est trop longue, divise-la en deux phrases courtes.
                    Voici le texte :\n\n {chunk}\n\n
                    Étapes :"""
                )
            }
        ],
        max_tokens=500,
        temperature=0.7
    )
    return completion.choices[0].message.content

def get_unique_filename(base_name):
    # Split the base name into name and extension
    name, ext = os.path.splitext(base_name)
    counter = 0
    new_name = base_name
    # Increment counter until a non-existing filename is found
    while os.path.exists(new_name):
        counter += 1
        new_name = f"{name}_{counter}{ext}"
    return new_name

def save_all_insights_to_json(output_file, insights):
    # Get a unique filename for the output file
    output_file = get_unique_filename(output_file)

    # Load existing data if the file exists
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = {}

    # Update the existing data with the new insights
    data.update(insights)

    # Save the updated data back to the file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    # Update the pointer file with the path to the latest JSON file
    pointer_file = "config.json"
    with open(pointer_file, "w", encoding="utf-8") as f:
        json.dump({"latest_file": output_file}, f, ensure_ascii=False, indent=4)



# Load environment variables from .env file
load_dotenv()

# Get the API key
openai.api_key = os.getenv("OPENAI_API_KEY")


# Initialize the "all_chunks" variable to store data
all_chunks = []

# Path to the folder containing text files
folder_path = "Book"

# Loop through all .txt files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        # Read the text file
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # Create the chunks for this file
        chunks = split_and_merge_text(text, min_size=1500, max_size=3000)

        # Get the title and author from the text
        title, author = extract_title_and_author(text)

        # Add chunks to all_chunks with their associated title and author
        for chunk in chunks:
            all_chunks.append({
                "title": title,
                "author": author,
                "chunk": chunk,
                "size": len(chunk),
            })


print("Lenght of all chunk : " + str(len(all_chunks)))


# Randomly select 20 chunks from "all_chunks"
random_chunks = random.sample(all_chunks, min(25, len(all_chunks)))

# Assuming all_chunks is already defined
titles_and_authors = [
    {"title": chunk["title"], "author": chunk["author"], "size": chunk["size"]}
    for chunk in random_chunks
]

# Display the unique titles, authors, and sizes
for item in titles_and_authors:
    print(f"Title: {item['title']}, Author: {item['author']}, Size: {item['size']} characters")

Lenght of all chunk : 2136
Title: Histoire du structuralisme, Tome 2, Author: François Dosse, Size: 2912 characters
Title: Histoire du structuralisme, Tome 2, Author: François Dosse, Size: 1989 characters
Title: Franz Kafka, Author: Bernard Lahire, Size: 2258 characters
Title: La question anthropologique (Cours 1954-1955), Author: Michel Foucault, Size: 1007 characters
Title: La question anthropologique (Cours 1954-1955), Author: Michel Foucault, Size: 2646 characters
Title: Franz Kafka, Author: Bernard Lahire, Size: 2456 characters
Title: La question anthropologique (Cours 1954-1955), Author: Michel Foucault, Size: 1926 characters
Title: Franz Kafka, Author: Bernard Lahire, Size: 1768 characters
Title: Histoire du structuralisme, Tome 2, Author: François Dosse, Size: 2531 characters
Title: Histoire du structuralisme, Tome 2, Author: François Dosse, Size: 2994 characters
Title: Histoire du structuralisme, Tome 1, Author: François Dosse, Size: 2791 characters
Title: Franz Kafka, Author:

In [37]:
print(random_chunks[0])

{'title': "La fin d'un grand partage", 'author': 'Pierre Charbonnier', 'chunk': 'Qu’est-ce donc que la direction ouest-est, dans la pensée indigène ? Celle que prennent le poisson-chandelle et le saumon, quand ils arrivent du large, chaque année, pour remonter les fleuves et s’enfoncer vers l’amont. Si cette orientation est aussi celle que doivent adopter les Tsimshian afin d’obtenir une image non déformée de leur existence sociale concrète, n’est-ce pas qu’ils se conçoivent sub specie piscis : qu’ils se mettent à la place des poissons, ou plutôt qu’ils mettent les poissons à leur place73 ?Le contexte sociologique tsimshian, en l’espèce le régime patrilocal, fournit ici le sens de lecture prioritaire du mythe, il donne une connotation positive à tout mouvement se déroulant dans le sens ouest-est. Dès lors que cette orientation ouest-est revêt le rôle d’axe directeur, sa superposition avec un événement aussi primordial pour la société tsimshian que le retour du poisson après une période

In [40]:
# Process each chunk and collect all insights
all_insights = {}
for i, chunk in enumerate(random_chunks, start=1):
    # Retrieve an insight from the chunk
    insight = retrieve_insight(chunk['chunk'], chunk['author'], chunk['title'])

    # Parse the insight into individual points
    points = [line.strip() for line in insight.strip().split('\n') if line]

    # Construct the JSON structure for this entry
    all_insights[str(i)] = {
        "Title and author": f"{chunk['title']}, {chunk['author']}",
        "Content": points
    }

# Save all collected insights into a single JSON file
save_all_insights_to_json("Insights/insights.json", all_insights)

In [25]:
import os

# Initialize the "all_chunks" variable to store data
all_chunks = []

# Path to the folder containing text files
folder_path = "Book"

# Loop through all .txt files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        # Read the text file
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # Create the chunks for this file
        chunks = split_and_merge_text(text, min_size=1500, max_size=3000)

        # Get the title and author from the text
        title, author = extract_title_and_author(text)

        # Add chunks to all_chunks with their associated title and author
        for chunk in chunks:
            all_chunks.append({
                title,
                author,
                chunk
            })

# Check the first few entries to confirm structure
all_chunks[:3]


[{'Bernard Lahire',
  'Franz Kafka, Chapitre 1',
  'Titre : Franz Kafka, Chapitre 1\nAuteur : Bernard Lahire\n\n\nChapitre 1\nL’enfermement dans le champLa seule manière de se défaire réellement de problèmes scientifiques, si l’on considère qu’une théorie sociologique est essentiellement un univers cohérent de problèmes-solutions articulés, c’est de les affronter, de les faire travailler, de les soumettre à examens, pour finalement les dépasser en découvrant leurs limites de validité et leur champ de pertinence. Partant d’une telle conception de la pratique scientifique, on ne peut qu’être d’accord avec l’analyse de Thomas S. Kuhn selon laquelle «\u2005seules les investigations fermement enracinées dans la tradition scientifique contemporaine ont une chance de briser cette tradition et de donner naissance à une nouvelle1\u2005». Et l’épistémologue ajoutait à la suite cette proposition d’une grande justesse : «\u2005Le savant productif doit être un traditionaliste qui aime à s’adonner à

In [12]:
# Extract title and author from the text
with open('Book/LOrdre matériel du savoir.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

title = None
author = None

# Extract title and author
for line in lines[:5]:  # Look at the first few lines for these fields
    if line.startswith("Titre :"):
        title = line.split(":", 1)[1].strip()
    elif line.startswith("Auteur :"):
        author = line.split(":", 1)[1].strip()
        
print(author)

print(title)


Françoise Waquet
L'ordre matériel du savoir
